In [1]:
import json
import requests
import shutil
import os
import federated_learning as fl
import feedernet
import pandas as pd

# Feedernet 회원 가입
1. Feedernet [회원 가입](https://qa.feedernet.kr/member/main)
1. 아래의 변수에 해당 회원 정보 기입

In [2]:
username = "fnet05@fnet.or.kr"
password = "123456"

project_id와 analysis_id얻기
1. 로그인 후 My Profile에서 기관 추가 - OpenDatabase 1,2,3,4 중 원하는 기관 선택: 연합학습을 원하면 2개이상 선택
1. 좌측 [My Project](https://qa.feedernet.kr/project)를 클릭하여 프로젝트 생성  --> project number 기록
1. 생성된 프로젝트에서 Analysis Tab을 선택한 후 Analysis 생성 --> analysis number 기록
1. 연구에 수행할 프로그램은 작성되어 있다고 가정

In [3]:
project_id = 145
analysis_id = 158

# Feedernet 로그인
1. Jupyter를 통해 Feedernet API를 호출할 수 있는 객체 생성
1. 생성된 객체로 Feedernet의 사용자 아이디, 암호를 사용하여 로그인

In [4]:
fn = feedernet.Feedernet()
fn.login(username, password)

login success


# CDM 선택
1. 위에서 생성한 객체를 통해 CDM조회
2. 사용할 CDM의 아이디 선택 ==> 연구 실행시 파라미터로 전달해야 함

In [5]:
data = fn.query_cdms(project_id, analysis_id)
cdm_df = pd.DataFrame(data, columns=['id','name', 'orgName', 'cdmVersion']).sort_values(by='orgName')
cdm_df

,id,name,orgName,cdmVersion
30,3,199_hira_2016,198 origin,5.3.0
29,23,TEST_SYNPUF,EvidNet_DQM,5.3.0
28,8,CDM_HIRA_2016,EvidNet_DQM,5.3.0
33,4,199_hira2016_from_204_18001,ORG204-18001,OMOP CDM v5.3.0
32,2,synpuf_in_204_18001,ORG204-18001,OMOP CDM v5.3.0
34,7,CDM_HIRA_2016,ORG204-18001,OMOP CDM v5.3.0
31,6,199_hira2016_from_204_18002,ORG204-18002,OMOP CDM v5.3.0
17,14,FNET1_HIRA2014,Open Database 1,OMOP CDM v5.3.0
16,13,FNET1_HIRA2013,Open Database 1,OMOP CDM v5.3.0
15,11,FNET1_HIRA2012,Open Database 1,OMOP CDM v5.3.0


In [6]:
cdms = [13, 22]

# Image 선택
1. 연구에 사용할 이미지 조회
- 연구에 사용할 이미지를 새로 만들기를 원하면 Spec을 Evidnet에 알려주어 이미지 생성을 요청한다
1. 사용할 이미지의 이름 준비 ==> 연구 실행시 파라미터로 전달해야 함

In [12]:
image_data = fn.query_images()
image_df = pd.DataFrame(image_data, columns=['image', 'description'])
pd.options.display.max_colwidth = 100
image_df

,image,description
0,docker.evidnet.co.kr/base-images/evidnet/tensorflow:latest,None


In [8]:
image = 1

# 연구 작성 파일 확인
1. 연구디렉토리를 생성
1. 연구디렉토리 내에 run.py파일을 생성한다.(실행 시잘 파일)
- run.py내에 모델을 실행하고, 결과(local_weight.json등)를 저장하는 구현이 되어야 한다.
1. 연구디렉토리 내에 global_weight.json이 있어야 한다.
1. 연구디렉토리를 기록한다.
1. 위에서 얻은 값(project id, analysis id, cdmid리스트, 연구 디렉토리)를 전달하며 FederatedLearning객체 생성
1. 객체 실행

In [10]:
research_location = "script"
f1 = fl.FederatedLearning(project_id, analysis_id, cdms, 3, image, research_location)
f1.login(username, password)
f1.execute()

login successful
starts round 0 


        ###################  Round 0 Starts ...   ###########################
        
####################script/global_weight_0.json
read_weight_from_json: len:  10 <class 'list'>
(1, 1024) <class 'numpy.ndarray'>
(1024,) <class 'numpy.ndarray'>
(1024, 512) <class 'numpy.ndarray'>
(512,) <class 'numpy.ndarray'>
(512, 256) <class 'numpy.ndarray'>
(256,) <class 'numpy.ndarray'>
(256, 128) <class 'numpy.ndarray'>
(128,) <class 'numpy.ndarray'>
(128, 1) <class 'numpy.ndarray'>
(1,) <class 'numpy.ndarray'>
make_zip_file :  /Users/jonggulpark/project/fl/federatedlearning
upload_analysis_file :  temp/run_0.zip  :::  temp/run_0.zip
API_UPDATE URL :  https://qa-api.feedernet.kr/api/project/145/analysis/158/researchFileForML?containerImage=docker.evidnet.co.kr/base-images/evidnet/tensorflow:latest
API_UPDATE : RESULT :  {'data': {'ok': 1}, 'token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1aWQiOjUsImxwbSI6MTU5ODQ4NTQzNDA0MywidHMiOjE2NjM2NTM4MDA0OTksImlhdCI6MT

API_RUN_STATUS : RESULT :  {'data': {'execution': {'id': 16250, 'analysisId': 158, 'createdAt': '2022-09-20T06:03:21.000Z', 'executionStatus': 11, 'executionStatusLast': None, 'uidCreator': 5, 'cdmId': 22, 'token': 'db851721d5cc435881fd4ac6ec5efd3a7cd378b548b14eebace465373deb', 'researchType': 4, 'updatedAt': '2022-09-20T06:03:23.000Z', 'rVersion': 'custom', 'clientId': None, 'containerId': None, 'startedAt': None, 'endedAt': None, 'errorMessage': None, 'resourceUsage': None, 'numOutputFiles': 0, 'name': 'M/L Tensorflow Analysis Test', 'engineVer': '0.2.1', 'atlasVersion': None}, 'files': [{'id': '3c0c36bbc875775e63047a33eca28062', 'version': 259, 'fileName': 'run_0.zip', 'mimeType': 'application/zip', 'analysisId': 158, 'fileId': 'c589ba77-b41f-4a64-a634-e16a458aa1d3', 'sha1': '068a866a83cd7dce786e2070b4022fcf34f2a029', 'createdAt': '2022-09-20T06:03:20.000Z', 'deletedAt': None, 'archived': 0, 'containerParam': None, 'image': 'docker.evidnet.co.kr/base-images/evidnet/tensorflow:latest

API_RUN_STATUS : RESULT :  {'data': {'execution': {'id': 16249, 'analysisId': 158, 'createdAt': '2022-09-20T06:03:21.000Z', 'executionStatus': 14, 'executionStatusLast': None, 'uidCreator': 5, 'cdmId': 13, 'token': 'dc2dee13c5f645768f15f89e4623f9546f2e6726e76541cf83029ff58740', 'researchType': 4, 'updatedAt': '2022-09-20T06:03:23.000Z', 'rVersion': 'custom', 'clientId': '5664', 'containerId': None, 'startedAt': '2022-09-20T06:03:48.000Z', 'endedAt': None, 'errorMessage': None, 'resourceUsage': None, 'numOutputFiles': 0, 'name': 'M/L Tensorflow Analysis Test', 'engineVer': '2.5.0', 'atlasVersion': 'ml'}, 'files': [{'id': '3c0c36bbc875775e63047a33eca28062', 'version': 259, 'fileName': 'run_0.zip', 'mimeType': 'application/zip', 'analysisId': 158, 'fileId': 'c589ba77-b41f-4a64-a634-e16a458aa1d3', 'sha1': '068a866a83cd7dce786e2070b4022fcf34f2a029', 'createdAt': '2022-09-20T06:03:20.000Z', 'deletedAt': None, 'archived': 0, 'containerParam': None, 'image': 'docker.evidnet.co.kr/base-images/e

API_RUN_STATUS : RESULT :  {'data': {'execution': {'id': 16250, 'analysisId': 158, 'createdAt': '2022-09-20T06:03:21.000Z', 'executionStatus': 14, 'executionStatusLast': None, 'uidCreator': 5, 'cdmId': 22, 'token': 'db851721d5cc435881fd4ac6ec5efd3a7cd378b548b14eebace465373deb', 'researchType': 4, 'updatedAt': '2022-09-20T06:03:23.000Z', 'rVersion': 'custom', 'clientId': '5665', 'containerId': None, 'startedAt': '2022-09-20T06:03:32.000Z', 'endedAt': None, 'errorMessage': None, 'resourceUsage': None, 'numOutputFiles': 0, 'name': 'M/L Tensorflow Analysis Test', 'engineVer': '2.5.0', 'atlasVersion': 'ml'}, 'files': [{'id': '3c0c36bbc875775e63047a33eca28062', 'version': 259, 'fileName': 'run_0.zip', 'mimeType': 'application/zip', 'analysisId': 158, 'fileId': 'c589ba77-b41f-4a64-a634-e16a458aa1d3', 'sha1': '068a866a83cd7dce786e2070b4022fcf34f2a029', 'createdAt': '2022-09-20T06:03:20.000Z', 'deletedAt': None, 'archived': 0, 'containerParam': None, 'image': 'docker.evidnet.co.kr/base-images/e

API_RUN_STATUS : RESULT :  {'data': {'execution': {'id': 16249, 'analysisId': 158, 'createdAt': '2022-09-20T06:03:21.000Z', 'executionStatus': 15, 'executionStatusLast': None, 'uidCreator': 5, 'cdmId': 13, 'token': 'dc2dee13c5f645768f15f89e4623f9546f2e6726e76541cf83029ff58740', 'researchType': 4, 'updatedAt': '2022-09-20T06:04:50.000Z', 'rVersion': 'custom', 'clientId': '5664', 'containerId': None, 'startedAt': '2022-09-20T06:03:48.000Z', 'endedAt': None, 'errorMessage': None, 'resourceUsage': None, 'numOutputFiles': 0, 'name': 'M/L Tensorflow Analysis Test', 'engineVer': '2.5.0', 'atlasVersion': 'ml'}, 'files': [{'id': '3c0c36bbc875775e63047a33eca28062', 'version': 259, 'fileName': 'run_0.zip', 'mimeType': 'application/zip', 'analysisId': 158, 'fileId': 'c589ba77-b41f-4a64-a634-e16a458aa1d3', 'sha1': '068a866a83cd7dce786e2070b4022fcf34f2a029', 'createdAt': '2022-09-20T06:03:20.000Z', 'deletedAt': None, 'archived': 0, 'containerParam': None, 'image': 'docker.evidnet.co.kr/base-images/e

...

API_RUN_STATUS : RESULT :  {'data': {'execution': {'id': 16253, 'analysisId': 158, 'createdAt': '2022-09-20T06:07:29.000Z', 'executionStatus': 11, 'executionStatusLast': None, 'uidCreator': 5, 'cdmId': 13, 'token': '1254f4180282402b98c54644534518ebc1a3aa7d159546e98c47e27dec0f', 'researchType': 4, 'updatedAt': '2022-09-20T06:07:29.000Z', 'rVersion': 'custom', 'clientId': None, 'containerId': None, 'startedAt': None, 'endedAt': None, 'errorMessage': None, 'resourceUsage': None, 'numOutputFiles': 0, 'name': 'M/L Tensorflow Analysis Test', 'engineVer': '0.2.1', 'atlasVersion': None}, 'files': [{'id': 'f0fc20c48aa3f99e90f44ba9585cd764', 'version': 261, 'fileName': 'run_2.zip', 'mimeType': 'application/zip', 'analysisId': 158, 'fileId': 'b5394585-579d-43f4-9cdc-aecac46185be', 'sha1': '2f4edb4f0ca81152116a2a026574030a603f1b7d', 'createdAt': '2022-09-20T06:07:28.000Z', 'deletedAt': None, 'archived': 0, 'containerParam': None, 'image': 'docker.evidnet.co.kr/base-images/evidnet/tensorflow:latest

API_RUN_STATUS : RESULT :  {'data': {'execution': {'id': 16254, 'analysisId': 158, 'createdAt': '2022-09-20T06:07:29.000Z', 'executionStatus': 14, 'executionStatusLast': None, 'uidCreator': 5, 'cdmId': 22, 'token': '04fa7e94561449eb90c5c77439837c0035f8e7af8d724bbe8d6e27a9aa01', 'researchType': 4, 'updatedAt': '2022-09-20T06:07:30.000Z', 'rVersion': 'custom', 'clientId': '5669', 'containerId': None, 'startedAt': '2022-09-20T06:07:38.000Z', 'endedAt': None, 'errorMessage': None, 'resourceUsage': None, 'numOutputFiles': 0, 'name': 'M/L Tensorflow Analysis Test', 'engineVer': '2.5.0', 'atlasVersion': 'ml'}, 'files': [{'id': 'f0fc20c48aa3f99e90f44ba9585cd764', 'version': 261, 'fileName': 'run_2.zip', 'mimeType': 'application/zip', 'analysisId': 158, 'fileId': 'b5394585-579d-43f4-9cdc-aecac46185be', 'sha1': '2f4edb4f0ca81152116a2a026574030a603f1b7d', 'createdAt': '2022-09-20T06:07:28.000Z', 'deletedAt': None, 'archived': 0, 'containerParam': None, 'image': 'docker.evidnet.co.kr/base-images/e

API_RUN_STATUS : RESULT :  {'data': {'execution': {'id': 16253, 'analysisId': 158, 'createdAt': '2022-09-20T06:07:29.000Z', 'executionStatus': 14, 'executionStatusLast': None, 'uidCreator': 5, 'cdmId': 13, 'token': '1254f4180282402b98c54644534518ebc1a3aa7d159546e98c47e27dec0f', 'researchType': 4, 'updatedAt': '2022-09-20T06:07:29.000Z', 'rVersion': 'custom', 'clientId': '5668', 'containerId': None, 'startedAt': '2022-09-20T06:07:54.000Z', 'endedAt': None, 'errorMessage': None, 'resourceUsage': None, 'numOutputFiles': 0, 'name': 'M/L Tensorflow Analysis Test', 'engineVer': '2.5.0', 'atlasVersion': 'ml'}, 'files': [{'id': 'f0fc20c48aa3f99e90f44ba9585cd764', 'version': 261, 'fileName': 'run_2.zip', 'mimeType': 'application/zip', 'analysisId': 158, 'fileId': 'b5394585-579d-43f4-9cdc-aecac46185be', 'sha1': '2f4edb4f0ca81152116a2a026574030a603f1b7d', 'createdAt': '2022-09-20T06:07:28.000Z', 'deletedAt': None, 'archived': 0, 'containerParam': None, 'image': 'docker.evidnet.co.kr/base-images/e

API_RUN_STATUS : RESULT :  {'data': {'execution': {'id': 16254, 'analysisId': 158, 'createdAt': '2022-09-20T06:07:29.000Z', 'executionStatus': 15, 'executionStatusLast': None, 'uidCreator': 5, 'cdmId': 22, 'token': '04fa7e94561449eb90c5c77439837c0035f8e7af8d724bbe8d6e27a9aa01', 'researchType': 4, 'updatedAt': '2022-09-20T06:08:43.000Z', 'rVersion': 'custom', 'clientId': '5669', 'containerId': None, 'startedAt': '2022-09-20T06:07:38.000Z', 'endedAt': None, 'errorMessage': None, 'resourceUsage': None, 'numOutputFiles': 0, 'name': 'M/L Tensorflow Analysis Test', 'engineVer': '2.5.0', 'atlasVersion': 'ml'}, 'files': [{'id': 'f0fc20c48aa3f99e90f44ba9585cd764', 'version': 261, 'fileName': 'run_2.zip', 'mimeType': 'application/zip', 'analysisId': 158, 'fileId': 'b5394585-579d-43f4-9cdc-aecac46185be', 'sha1': '2f4edb4f0ca81152116a2a026574030a603f1b7d', 'createdAt': '2022-09-20T06:07:28.000Z', 'deletedAt': None, 'archived': 0, 'containerParam': None, 'image': 'docker.evidnet.co.kr/base-images/e

API_GET_RESULT_LIST : RESULT :  {'data': {'list': [{'id': '630e6723-f840-4e4d-81a3-f26c7769c42d', 'fileName': 'result.zip/data/results/log.txt', 'mimeType': 'text/plain', 'createdAt': '2022-09-20T06:09:11.000Z', 'deletedAt': None, 'sha1': '066045b9061516bf67a783d027b4b62c03575c30', 'executionId': 16253, 'resultFileType': 2}, {'id': 'd2172547-a71c-4749-97d0-20eb5e4f0040', 'fileName': 'result.zip', 'mimeType': 'application/zip', 'createdAt': '2022-09-20T06:09:09.000Z', 'deletedAt': None, 'sha1': 'cbbb77c6c0f6fcd781c975226574ad2cf152a3e8', 'executionId': 16253, 'resultFileType': 1}, {'id': '6523db92-ef2a-4ccc-b758-70194acf0f99', 'fileName': 'result.zip/data/results/local_weight.json', 'mimeType': 'application/json', 'createdAt': '2022-09-20T06:09:05.000Z', 'deletedAt': None, 'sha1': 'e29cee292fe0490db694cc5e817b2ef5a5244ac7', 'executionId': 16253, 'resultFileType': 2}, {'id': '431ca4a2-23a4-4224-9515-51b17f454a7d', 'fileName': 'result.zip/data/results/scores.csv', 'mimeType': 'text/csv', 

In [ ]:
# 결과 확인
1. 연구디렉토리/output디렉토리에 round별 local_weight_XXXXX.json값이 저장됨
1. 연구디렉토리에 round별 global_weight_{round}.json이 저장됨